In [39]:
import pyautogui
import pandas as pd
df = pd.DataFrame(columns=['num', 'top', 'left', 'width', 'height'])
img_path = './imgs/'

ll = 0
for i in range(1, 9+1):
    positions = pyautogui.locateAllOnScreen(f'{img_path}{i}.png', confidence=0.90)
    for pos in positions:
        new_row = {'num':i, 'top':pos.top, 'left':pos.left, 
                   'width':pos.width, 'height':pos.height}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        ll += 1

In [40]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"
os.environ["OMP_NUM_THREADS"] = "2"

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=170)
kmeans.fit(df[['top', 'left']])
df['cluster'] = kmeans.labels_
df = df.groupby(by='cluster').mean()

In [41]:
kmeans = KMeans(n_clusters=17)
kmeans.fit(df[['left']])
df['x_cls'] = kmeans.labels_

kmeans = KMeans(n_clusters=10)
kmeans.fit(df[['top']])
df['y_cls'] = kmeans.labels_

c:\Users\zerot\anaconda3\envs\practice\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\zerot\anaconda3\envs\practice\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [42]:
for xi in range(17):
    mean_value = df.loc[df['x_cls'] == xi, 'left'].mean()  # 먼저 평균값 계산
    df.loc[df['x_cls'] == xi, 'left'] = mean_value  # loc를 사용하여 값 업데이트
for xi in range(10):
    mean_value = df.loc[df['y_cls'] == xi, 'top'].mean()  # 먼저 평균값 계산
    df.loc[df['y_cls'] == xi, 'top'] = mean_value  # loc를 사용하여 값 업데이트

In [43]:
import numpy as np
df = df.sort_values(by=['top', 'left']).reset_index(drop=True)
num1d = np.array(df['num'].values.tolist(), dtype=int)
num2d = num1d.reshape(10, 17)

In [44]:
import pyautogui
import keyboard
xloc = df['left'].unique()
yloc = df['top'].unique()

dx = np.mean(np.diff(xloc))
dy = np.mean(np.diff(yloc))
xloc_mouse = xloc - 0.25 * dx
yloc_mouse = yloc - 0.25 * dy

def mouse_move(xi, yi, w, h, duration):
    pyautogui.moveTo( xloc_mouse[xi], yloc_mouse[yi], duration=duration)
    pyautogui.mouseDown()
    pyautogui.moveTo(xloc_mouse[xi] + (w+1)*dx, yloc_mouse[yi] + (h+1)*dy,  duration=duration)
    pyautogui.moveTo(xloc_mouse[xi] + (w+1)*dx, yloc_mouse[yi] + (h+1)*dy,  duration=duration)
    pyautogui.mouseUp()

duration = 0.01
for _ in range(10):
    for yi in range(10):
        for xi in range(17):
            if keyboard.is_pressed("q"):  
                break
            w_max, h_max = 17 - xi, 10 - yi
            for h in range(h_max):
                for w in range(w_max):
                    if np.sum(num2d[yi: yi + h+1, xi: xi + w+1]) == 10:
                        num2d[yi: yi + h+1, xi: xi + w+1] = 0
                        mouse_move(xi, yi, w, h, duration)


